**Импорты**

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
%run lab4.ipynb
%run lab5.ipynb
%run lab2.ipynb
%run lab3.ipynb

Кол-во признаков: 4
(135, 4) (135,)
Точность: 100.00%
Accuracy: 100.00%
Кол-во признаков: 4
Количество кластеров: 3
Кластер 1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Кластер 2: [118, 50, 51, 52, 100, 102, 104, 105, 107, 108, 109, 112, 116, 117, 120, 122, 124, 125, 128, 129, 130, 131, 132, 135, 136, 139, 140, 141, 143, 144, 145]
Кластер 3: [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 103, 106, 110, 111, 113, 114, 115, 119, 121, 123, 126, 127, 133, 134, 137, 138, 142, 146, 147, 148, 149]
Метки:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

**Оценка классификаторов**

In [5]:
import pandas as pd

def accuracy(y_pred, y_true):
    return np.mean(y_pred == y_true)

def precision(y_pred, y_true):
    t_p = np.sum((y_pred == 1) & (y_true == 1))
    f_p = np.sum((y_true == -1) & (y_pred == 1))
    return t_p / (t_p + f_p) if (t_p + f_p) > 0 else 0

def recall(y_pred, y_true):
    t_p = np.sum((y_pred == 1) & (y_true == 1))
    f_n = np.sum((y_pred == -1) & (y_true == 1))
    return t_p / (t_p + f_n) if (t_p + f_n) > 0 else 0

def f1(y_pred, y_true):
    pre = precision(y_pred, y_true)
    re = recall(y_pred, y_true)
    return 2 * (pre * re) / (pre + re) if (pre + re) > 0 else 0


def mse(y_pred, y_true):
    return np.mean((y_true - y_pred) ** 2)

def r_squared(y_pred, y_true):
    return 1 - (mse(y_pred, y_true) / mse(np.mean(y_true), y_true))

def confusion_matrix(y_pred, y_true):
    unique_labels = np.unique(y_true)
    matrix = pd.DataFrame(data=np.zeros((len(unique_labels), len(unique_labels)), dtype=int), columns = np.unique(y_true), index = np.unique(y_true))
    #matrix = np.zeros((len(unique_labels), len(unique_labels)), dtype=int)
    for y_t, y_p in zip(y_true, y_pred):
        matrix.loc[y_t, y_p] += 1
    
    return matrix

def classifiaction_metrics(y_pred, y_true, twoClasses=True):
    s = []
    metrics = [accuracy, precision, recall, f1, mse, r_squared, confusion_matrix] if twoClasses else [accuracy, mse, r_squared, confusion_matrix]
    for metric in metrics:
        s.append(metric(y_pred, y_true))
    return s

#### **SVM**

In [87]:
iris = load_iris()
X = iris.data
y = iris.target
X = X[y != 2]
y = y[y != 2]
y = np.where(y == 0, -1, 1)

metrics = []
for _ in range(10): # кросс-валидация
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    svm = SVM(learning_rate=0.1, lambda_param=0.1, n_iters=100)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    metrics.append(classifiaction_metrics(y_pred, y_test))

df = pd.DataFrame(data=metrics, columns=['acc', 'pre', 're', 'f1', 'mse', 'r squared', 'con'])


print(df.describe().loc['mean'])
for sample in metrics:
    print(sample[-1])

acc          1.0
pre          1.0
re           1.0
f1           1.0
mse          0.0
r squared    1.0
Name: mean, dtype: float64
    -1   1
-1  14   0
 1   0  16
    -1   1
-1  14   0
 1   0  16
    -1   1
-1  14   0
 1   0  16
    -1   1
-1  15   0
 1   0  15
    -1   1
-1  16   0
 1   0  14
    -1   1
-1  13   0
 1   0  17
    -1   1
-1  16   0
 1   0  14
    -1   1
-1  15   0
 1   0  15
    -1   1
-1  18   0
 1   0  12
    -1   1
-1  14   0
 1   0  16


In [3]:
iris = load_iris()
X = iris.data
y = iris.target


#### **KNN**

In [89]:
metrics = []
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
    y_pred = np.array([knn(X_train, y_train, x_test, 5) for x_test in X_test])
    metrics.append(classifiaction_metrics(y_pred, y_test, twoClasses=False))


df = pd.DataFrame(data=metrics, columns=['acc', 'mse', 'r squared', 'con'])


print(df.describe().loc['mean'])
print('confusion matrixes:')
for sample in metrics:
    print(sample[-1])

acc          0.96000
mse          0.04000
r squared    0.94046
Name: mean, dtype: float64
confusion matrixes:
   0  1  2
0  6  0  0
1  0  3  1
2  0  0  5
   0  1  2
0  7  0  0
1  0  5  0
2  0  0  3
   0  1  2
0  6  0  0
1  0  6  0
2  0  0  3
   0  1  2
0  7  0  0
1  0  2  0
2  0  1  5
   0  1  2
0  7  0  0
1  0  5  0
2  0  0  3
   0  1  2
0  3  0  0
1  0  5  0
2  0  0  7
   0  1  2
0  5  0  0
1  0  5  1
2  0  0  4
   0  1  2
0  5  0  0
1  0  4  1
2  0  0  5
   0  1  2
0  4  0  0
1  0  5  1
2  0  0  5
   0  1  2
0  6  0  0
1  0  4  1
2  0  0  4


**Оценка кластеризации**

In [6]:
def silhouette(X, centers, labels):
    score = []
    for i, x_test in enumerate(X):
        a = np.mean([euclidean_distance(x_test, x_same) for j, x_same in enumerate(X) if labels[i] == labels[j] and j != i])
        d = [euclidean_distance(x_test, x_other) for j, x_other in enumerate(X)]
        min = float('inf')
        k = 0
        for z, x in enumerate(d):
            if labels[z] == labels[i]:
                continue
            if x < min:
                min = x
                k = z
        b = np.mean([dis for i, dis in enumerate(d) if labels[k] == labels[i]])
        score.append((b - a) / max(a, b) if max(a, b) > 0 else 0)
    return np.mean(score)

def di(X, centers, labels):
    other_min = np.min([euclidean_distance(x_test, x_other) for j, x_other in enumerate(X)
                        for i, x_test in enumerate(X) if labels[i] != labels[j]])
    same_max = np.max([euclidean_distance(x_test, x_same) for j, x_same in enumerate(X)
                       for i, x_test in enumerate(X) if labels[i] == labels[j] and j != i])
    return other_min / same_max
    
def dbi(X, centers, labels):
    c = []
    for i in range(len(centers)):
        same_dis = np.sum([euclidean_distance(centers[i], x)
                           for j, x in enumerate(X) if labels[j] == i]) / np.linalg.norm(centers[i])
        other_dis = []
        between_dis = []
        for j in range(len(centers)):
            if j != i:
                other_dis.append(np.sum([euclidean_distance(centers[j], x)
                                         for z, x in enumerate(X) if labels[z] == j]) / np.linalg.norm(centers[j]) + same_dis)
                between_dis.append(euclidean_distance(centers[i], centers[j]))
        c.append(np.max(other_dis) / np.min(between_dis))
    
    return np.mean(c)

def dbcv(X, centers, labels):
    c = []
    for i, ix in enumerate(X):
        max_d_other = np.max([euclidean_distance(ix, jx) for j, jx in enumerate(X) if labels[i] != labels[j]])
        for j, jx in enumerate(X):
            if i != j:
                c.append(euclidean_distance(ix, jx) / max_d_other)

    return np.mean(c)

def cluster_metrics(X, labels, centers):
    return [ m(X, centers, labels) for m in [silhouette, di, dbi, dbcv]]



### Максимин и k-средних

In [7]:
# test 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
k = 3

_, cluster_centers, labels = maximin_clustering(X, threshold=k)
print('maxmin')

metrics = []
metrics.append(classifiaction_metrics(labels, y, twoClasses=False) + cluster_metrics(X, labels, cluster_centers))
df = pd.DataFrame(data=metrics, columns=['acc', 'mse', 'r squared', 'con', 'sil', 'di', 'dbi', 'dbcv'])


print(df.drop(['con'], axis=1))
print('confusion matrixes:')
for sample in metrics:
    print(sample[3])

labels, centers = k_means(X, k)
print('k-means')

metrics = []
metrics.append(classifiaction_metrics(labels, y, twoClasses=False) + cluster_metrics(X, labels, centers))
df = pd.DataFrame(data=metrics, columns=['acc', 'mse', 'r squared', 'con', 'sil', 'di', 'dbi', 'dbcv'])


print(df.drop(['con'], axis=1))
print('confusion matrixes:')
for sample in metrics:
    print(sample[3])


maxmin
   acc  mse  r squared       sil        di       dbi      dbcv
0  0.5  0.5       0.25  0.522804  0.046575  4.458009  0.479025
confusion matrixes:
    0   1   2
0  50   0   0
1   0   3  47
2   0  28  22
k-means
        acc       mse  r squared       sil        di      dbi      dbcv
0  0.893333  0.106667       0.84  0.553454  0.098807  4.31806  0.479025
confusion matrixes:
    0   1   2
0  50   0   0
1   0  48   2
2   0  14  36
